In [1]:
import pandas as pd
import os
import glob
from datetime import datetime

In [2]:
path = os.getcwd() + "\\Dataset\\"
files = glob.glob(path + "/*precios*")

In [96]:
df_precios = pd.read_excel(files[0], dtype=object ,sheet_name=1)
df_precios1 = pd.read_excel(files[0], sheet_name=0)
df_precios2 = pd.read_csv(files[1],sep=',',encoding='utf-16')
df_precios3 = pd.read_json(files[2])

In [175]:
df_precios1 = pd.read_excel(files[0], sheet_name=0)

In [192]:
def unDate(x):
    """read_excel me trae algunas sucursal_id como si fueran datetime. Esta funcion toma elementos, chequea si son datetime y en caso de serlo, los convierte a str con el formato acorde al orden de identificadores de sucursal"""
    if isinstance(x,datetime):
        x=x.strftime("%d-%m-%Y")
    return x

def nasDups(df):
    """Cosas básicas: remover duplicados y nulos"""
    df = df.copy()
    df = df.dropna()
    df = df.drop_duplicates()
    return df

def typeCheck(df):
    if df.precio.dtype != "float":
        try:
            df.precio = df.precio.apply(lambda x: None if x == '' else x)
            df.precio = df.precio.astype(float)
        except:
            print("Error 1")
    if df.sucursal_id.dtype != "str":
        try:
            df.sucursal_id = df.sucursal_id.astype('string')
        except:
            print('Error str')
            
    if df.producto_id.dtype == "float":
        df.producto_id = df.producto_id.astype(int)
        df.producto_id = df.producto_id.astype('string')
    return df

def splitProd(df):
    df.producto_id = df.producto_id.str.split('-').str[-1]
    return df


In [156]:
df_precios = nasDups(df_precios)
df_precios = splitProd(df_precios)
df_precios = typeCheck(df_precios)

In [193]:
def etl(df):
    df = nasDups(df)
    df = typeCheck(df)
    df = splitProd(df)
    df = unDate(df)
    return df

In [195]:
df_precios1 = etl(df_precios1)
df_precios2 = etl(df_precios2)

In [194]:
df_precios3 = etl(df_precios3)

In [187]:
df_precios3.precio

0           29.9
1           39.9
2         499.99
3         539.99
4         519.99
           ...  
397600       409
397601       509
397602    114.99
397603       219
397604    349.99
Name: precio, Length: 22466, dtype: object

In [ ]:
df_precios1.producto_id = df_precios1.producto_id

In [ ]:
df_precios = etl(df_precios)
df_precios1 = etl(df_precios1)
df_precios2 = etl(df_precios2)
df_precios3 = etl(df_precios3)

In [188]:
df_precios1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 463753 entries, 0 to 478908
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   precio       463753 non-null  float64
 1   producto_id  463753 non-null  object 
 2   sucursal_id  463753 non-null  string 
dtypes: float64(1), object(1), string(1)
memory usage: 14.2+ MB


In [134]:
df_precios1[df_precios1.precio == df_precios1.precio.max()]

,precio,producto_id,sucursal_id
22476,4.399890e+16,NaN,36-3-25
22488,4.399890e+16,NaN,36-3-25
22489,4.399890e+16,NaN,36-3-25
22783,4.399890e+16,NaN,36-5-73


In [116]:
df_precios.precio.astype(float)

11125     829.00
11126      59.00
11127      59.00
11128       7.49
11129      59.00
           ...  
458382    145.00
458383    149.00
458384    509.00
458385     90.99
458386    349.99
Name: precio, Length: 12464, dtype: float64

Algunos precios son un str vacio: ''. Los convertimos a nulos para poder remover esos registros

In [191]:
df_precios3[df_precios3.precio == '']

,precio,producto_id,sucursal_id
8754,,0643131504438,65-1-317
9121,,0655257738062,65-1-317
9122,,0655257738246,65-1-317
28020,,0000000000048,65-1-317
28021,,0000000000242,65-1-317
...,...,...,...
28112,,0000000017254,65-1-317
28113,,0000000017257,65-1-317
28114,,0000000017419,65-1-317
28115,,0000000017420,65-1-317


In [87]:
df_precios3.precio = df_precios3.precio.apply(lambda x: None if x == '' else x)

In [77]:
df_precios.sucursal_id = df_precios['sucursal_id'].apply(unDate)

Vemos que hay nulos en precio y en producto_id.  
Los producto_id están como float, vamos a convertirlos

In [79]:
df_precios.producto_id =  df_precios['producto_id'].apply(lambda x: int(x) if isinstance(x,float) else None)
# df_precios.precio = df_precios.precio.astype(float)

In [89]:
df_precios2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 472131 entries, 0 to 472165
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   precio       472131 non-null  float64
 1   producto_id  472131 non-null  object 
 2   sucursal_id  472131 non-null  object 
dtypes: float64(1), object(2)
memory usage: 30.5+ MB


In [6]:
df_precios2 = nasDups(df_precios2)

In [39]:
df_precios.precio = df_precios.precio.astype(float)

In [129]:
df_precios2 = preProc(df_precios2)

In [144]:
count = 0
count_str = 0
for item in df_precios.producto_id:
    if isinstance(item,str):
        if '-' in item:
            count_str +=1
    else:
        count+=1
